## 3.1 构造决策树

决策树简单来说就是对英语某个数据集的一个流程图。数据集有很多的属性，选取属性的顺序不同，走的决策树顺序不一样，这个过程我们称之为划分数据集。

为了解决划分数据集的问题，我们引入了信息熵的概念。

### 3.1.1 信息熵

熵定义为信息的期望值，让我们先看一下信息量的定义，如果一件事情发生的概率小，那么这件事情发生代表的信息量就大；反之信息量就小，公式如下：
$$
l(x_i) = -log_2p(x_i)
$$

>上述公式描述的是，概率和信息量之间的变化关系，但是：
>
>
>- [为什么是对数函数？](https://zhuanlan.zhihu.com/p/26486223)
>
> 信息量的大小$l(x_i)$与事件的概率$p(x_i)$有关，而且如果两件相互独立的事件，同时发生，那么我们获得信息量为：(其中$l$代表信息量，$f$代表未知的函数)
>$$
 l(x_i)+l(x_j)=f(p(x_i))+f(p(x_j))=f(p(x_i)p(x_j))
$$
>
>
>   我们容易得出，函数$f$是一个与对数函数相关的函数。
>
>- 为什么是负数？
>
> $$
  l(x_i) = -{log}_2p(x_i)={log}_2p(x_i)^{-1}={log}_2\frac{1}{p(x_i)}
 $$
>
> 其实负数表达的是概率越小，会导致信息量越大。
>
>- 为什么底数为2？
>
> 一般来说是约定，当为2时我们求得的信息量的单位为比特, 取$ln$对应信息量的单位为奈特。

有了信息量的定义公示，那么我们可以很容易得出熵的公式，因为熵代表的就是信息的期望值，所以用事件发生的概率乘以信息量，再全部求和：


## 3.2 代码实现 

In [11]:
# -*- coding: UTF-8 -*-
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from six import StringIO
from sklearn import tree
import pandas as pd
import numpy as np
import pydotplus


In [ ]:
with open('data/Ch03/lenses.txt', 'r') as fr:                                        #加载文件
    lenses = [inst.strip().split('\t') for inst in fr.readlines()]        #处理文件
lenses_target = []                                                        #提取每组数据的类别，保存在列表里
for each in lenses:
    lenses_target.append(each[-1])
# print(lenses_target)

lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']            #特征标签        
lenses_list = []                                                        #保存lenses数据的临时列表
lenses_dict = {}                                                        #保存lenses数据的字典，用于生成pandas
for each_label in lensesLabels:                                            #提取信息，生成字典
    for each in lenses:
        lenses_list.append(each[lensesLabels.index(each_label)])
    lenses_dict[each_label] = lenses_list
    lenses_list = []
# print(lenses_dict)                                                        #打印字典信息
lenses_pd = pd.DataFrame(lenses_dict)                                    #生成pandas.DataFrame
# print(lenses_pd)                                                        #打印pandas.DataFrame
le = LabelEncoder()                                                        #创建LabelEncoder()对象，用于序列化            
for col in lenses_pd.columns:                                            #序列化
    lenses_pd[col] = le.fit_transform(lenses_pd[col])
# print(lenses_pd)                                                        #打印编码信息

clf = tree.DecisionTreeClassifier(max_depth = 4)                        #创建DecisionTreeClassifier()类
clf = clf.fit(lenses_pd.values.tolist(), lenses_target)                    #使用数据，构建决策树

dot_data = StringIO()
tree.export_graphviz(clf, out_file = dot_data,                            #绘制决策树
                    feature_names = lenses_pd.keys(),
                    class_names = clf.classes_,
                    filled=True, rounded=True,
                    special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("tree.pdf")                                                #保存绘制好的决策树，以PDF的形式存储。

print(clf.predict([[1,1,1,0]]))                                            #预测